In [3]:
import time
import pickle
import random

from selenium import webdriver
# from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.firefox.options import Options
from tqdm.auto import tqdm, trange
from fake_useragent import UserAgent

In [26]:
# fp = webdriver.FirefoxProfile()
# options = Options()
# options.headless = True
# fp.set_preference("network.cookie.cookieBehavior", 2)
    
# crawl_interval = 10.0
# driver = webdriver.Firefox(firefox_profile=fp)
driver = webdriver.Firefox()
driver.implicitly_wait(15)

In [29]:
# drivers = []
for i in range(30):
    fp.set_preference("general.useragent.override", UserAgent().random)
    fp.update_preferences()
    driver = webdriver.Firefox(firefox_profile=fp, options=options)
    driver.implicitly_wait(30)
    drivers.append(driver)

In [279]:
driver.get("https://movie.douban.com/tag/#/?sort=U&range=0,10&tags=%E5%8A%A8%E7%94%BB,%E7%94%B5%E5%BD%B1")

In [280]:
more_button = driver.find_element_by_class_name("more")
for i in trange(99):
    more_button.click()
    driver.execute_script("arguments[0].scrollIntoView();", more_button)
    time.sleep(7)

In [281]:
items = driver.find_elements_by_class_name("item")
len(items)

2000

In [282]:
animated_file_listing = []
for item in tqdm(items):
    animated_file_listing.append({'title': item.find_element_by_class_name("title").text,
                                  'page_link': item.get_attribute("href"),
                                  'rating': item.find_element_by_class_name("rate").text,
                                  'cover': item.find_element_by_tag_name("img").get_attribute("src")})

In [7]:
with open("animated_film_listing.pickle", "rb") as file:
    animated_film_listing = pickle.load(file)

In [34]:
for film in tqdm(animated_film_listing[126:]):
    driver = random.choice(drivers)
    
    driver.get(film['page_link'])
#     page_load_time = time.time()
    
    # fetch all info in plain text
    film_info = driver.find_element_by_id("info")
    info_dict = dict([line.split(": ", 1) for line in film_info.text.split("\n")])
    film.update(info_dict)
    film['全名'] = driver.find_element_by_css_selector("span[property='v:itemreviewed']").text
    film['年份'] = driver.find_element_by_class_name("year").text
    film['剧情简介'] = driver.find_element_by_css_selector("span[property='v:summary']").text
    
    
    # expand actors list
    try:
        film_info.find_element_by_class_name("more-actor").click()
    except:
        pass
    
    casts = []
    for actor in film_info.find_elements_by_css_selector("a[rel='v:starring']")[:10]:
        name = actor.text
        page_link = actor.get_attribute("href")
        casts.append({'name': name, 'page_link': page_link})
        
    film['主演'] = casts
    
    reviews = []
    for item in driver.find_elements_by_class_name("review-item")[:10]:
        review = {}
        # expand the current review
        expand_button = item.find_element_by_class_name("unfold")
        driver.execute_script("arguments[0].scrollIntoView();", expand_button)
        try:
            expand_button.click()
        except ElementClickInterceptedException:
            driver.execute_script("arguments[0].scrollIntoView();", expand_button)
            expand_button.click()
        except ElementNotInteractableException:
            continue
        
#         time.sleep(0.3)
        review['id'] = item.get_attribute("id")
        review['user'] = item.find_element_by_class_name("name").text
        review['time'] = item.find_element_by_class_name("main-meta").text
        review['text'] = item.find_element_by_class_name("clearfix").text
        reviews.append(review)
        
    film['影评'] = reviews
    driver.delete_all_cookies()
#     processing_time = time.time() - page_load_time
#     if processing_time < crawl_interval:
#         time.sleep(crawl_interval - processing_time)

NoSuchElementException: Message: Unable to locate element: [id="info"]


In [33]:
animated_film_listing[126:127]

[{'title': '名侦探柯南：沉默的十五分钟',
  'page_link': 'https://movie.douban.com/subject/6007281/',
  'rating': '6.7',
  'cover': 'https://img9.doubanio.com/view/photo/s_ratio_poster/public/p981385980.jpg',
  '导演': '静野孔文',
  '编剧': '古内一成 / 青山刚昌',
  '主演': [{'name': '高山南',
    'page_link': 'https://movie.douban.com/celebrity/1048421/'},
   {'name': '山崎和佳奈',
    'page_link': 'https://movie.douban.com/celebrity/1003697/'},
   {'name': '小山力也',
    'page_link': 'https://movie.douban.com/celebrity/1038087/'},
   {'name': '山口胜平',
    'page_link': 'https://movie.douban.com/celebrity/1018954/'},
   {'name': '草尾毅', 'page_link': 'https://movie.douban.com/celebrity/1042356/'},
   {'name': '林原惠美',
    'page_link': 'https://movie.douban.com/celebrity/1002915/'},
   {'name': '松井菜樱子',
    'page_link': 'https://movie.douban.com/celebrity/1003728/'},
   {'name': '岩居由希子',
    'page_link': 'https://movie.douban.com/celebrity/1015949/'},
   {'name': '高木涉', 'page_link': 'https://movie.douban.com/celebrity/1025675/'},
   

In [ ]:
with open("animated_film_2000.pickle", "wb") as file:
    pickle.dump(animated_film_listing, file)

In [35]:
for driver in drivers:
    driver.close()

InvalidSessionIdException: Message: Tried to run command without establishing a connection
